# Artificial Neural Network (ANN)
By: Alexander Peltier, Parker Spaan and Matthew Powers

## Data Preprocessing

In [25]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the data from CSV file
data_unclean = pd.read_csv('nba_stats.csv')

# Now filter the data for the seasons from 2016-17 to 2021-22
filtered_data = data_unclean[data_unclean['season'].between('2016-17', '2021-22')]

# Get 100 random samples from the filtered data
data = filtered_data.sample(100)

## Extract required columns

In [26]:
players = data.iloc[:, 1]  # Player names
stats = data.iloc[:, 12:15]  # Player stats (points, rebounds, assists)

## Definition of "Optimal Team"

#### We have defined our optimal team as the top 5 players of this player_values column
#### The ANN will be predicting these player_values and then we will take the top 5

#####           Note: we added this column to give the ANN somehting to predict that we could evaluate

##### We are using the players stats of points, rebounds, and assists for this metric and stating that points are the most valuable at a weight of 0.5, followed by rebounds at 0.3 and then assists at 0.2

#### The goal is to have the ANN derive these values and the dot product of them

In [27]:
# Define player value as a weighted sum of their stats
weights = [0.5, 0.3, 0.2]
player_values = stats.dot(weights)

## Build a MLP

#### The below is splitting up the training and testing set (20 players of the 100). It is then getting up hidden layers that are 100, 50, 25 and will itterate (epochs) of at most 1000. We choose the seed of 42 to make certain this can be reproduced again. The MLPRegressor is then able to go through and find the best fit (with back propigation). The training and testing data is then put in the MLP to find the most opitimal team

In [28]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(stats, player_values, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the MLP
mlp = MLPRegressor(hidden_layer_sizes=(100, 50, 25), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(100, 50, 25), max_iter=1000, random_state=42)

## Get The Results

In [29]:
# Predict player values using the trained model
predicted_values = mlp.predict(scaler.transform(stats))

print("#############################################")
# Rank the players based on predicted values and select the top 5
top_players_indices_predicted = predicted_values.argsort()[-5:][::-1]
top_players_predicted = players.iloc[top_players_indices_predicted]

print("Optimal Team Predicted:")
print(top_players_predicted)

print("#############################################")

# Rank the players based on actual values and select the top 5
top_players_indices_actual = player_values.argsort().iloc[-5:][::-1]
top_players_actual = players.iloc[top_players_indices_actual]

print("Optimal Team Actual:")
print(top_players_actual)
print("#############################################")

#############################################
Optimal Team Predicted:
12253         Nikola Jokic
10005         LeBron James
10284         Devin Booker
9662         DeMar DeRozan
12011    Russell Westbrook
Name: player_name, dtype: object
#############################################
Optimal Team Actual:
12253         Nikola Jokic
10005         LeBron James
10284         Devin Booker
9662         DeMar DeRozan
12011    Russell Westbrook
Name: player_name, dtype: object
#############################################


##### In Summary, the optimal predicted team based on the ANN player values is based on the above definitions. The team is created by going through the 100 randomly chosen players. With all 100 of these players their stats of points, rebounds, and assists all hold weights. This was utilized as to win the most games teams need to put up the most points. In addition to holding the ball for the majority of the time (thus rebounding) and finally players need to be able to move the ball around (assists). This assignment has tested us with applying a machine model to a real world problem. This was challenging as the most "optimal" team can have multiple different meanings. But this route was chosen as this is believed to be the most WINNING team based on evidence of how teams choose players in the real world. 

##### The ANN model utilized the 100 players to train itself on the values for each player; then with the dot product of the weights (how much we care about each statistic) was able to find the optimal players that will go together. Making sure we are not simply taking ONLY the highest scoring players but additionally the best rounded players. Chemistry of players is not possible with the data provided this is based ONLY on court play statistics. Scouts are for chemistry, ML models are for optimization. 

##### The reason we know this is that the players true values also agreed wth this conclusion as shown above when displaying the predicted and actual optimal teams. This is utilizing a 100, 50, 25 Neural network (initial 100, hidden 50, output 25).

##### While this 100% accuracy may seem too good to be true, we can verify that it is and that the result of this kind of accuracy is a result of the way we defined the problem and the weights, this mdel is simply converging on the unkown weights similar to that of a model converging onto the sin() formula or anything else of that nature